###  MN-DS data

In [2]:
# Read embeddings 
import pandas as pd
df=pd.read_csv("MNDS_embeddings.csv",index_col=0) #Web scrapping articles 
df.head()

(10880, 16)


,id,date,source,title,content,author,url,published,published_utc,collection_utc,category_level_1,category_level_2,Language,Keywords,Scores,Embeddings
data_id,,,,,,,,,,,,,,,,
1809,abcnews--2019-10-31--Virginia mom charged with...,2019-10-31,abcnews,Virginia mom charged with murder in 2-year-old...,the virginia woman whose year old son was fou...,NaN,https://abcnews.go.com/US/wireStory/virginia-m...,"Thu, 31 Oct 2019 16:49:56 -0400",1572554996,1572559512,"crime, law and justice",crime,en,"['murder', 'fractures', 'warrant', 'jailed', '...","[0.3616, 0.3054, 0.3039, 0.3011, 0.2889]",[[ 0.0559812 0.04057256 0.01520373 ... 0.0...
1980,abcnews--2019-11-07--2 escaped murder suspects...,2019-11-07,abcnews,2 escaped murder suspects arrested at US-Mexic...,authorities are trying to determine if anyone ...,NaN,https://abcnews.go.com/US/wireStory/escaped-mu...,"Thu, 07 Nov 2019 00:13:12 -0500",1573103592,1573131986,"crime, law and justice",crime,en,"['inmates', 'border', 'mexican', 'salinas', 'c...","[0.3711, 0.3435, 0.3249, 0.3249, 0.3033]",[[ 0.02810522 0.05049188 -0.00844738 ... 0.0...
1995,abcnews--2019-11-07--Family turns in escaped b...,2019-11-07,abcnews,"Family turns in escaped boy, 13, suspected in ...",a year old suspect in a double homicide who e...,NaN,https://abcnews.go.com/US/wireStory/family-tur...,"Thu, 07 Nov 2019 07:39:54 -0500",1573130394,1573131982,"crime, law and justice",crime,en,"['suspect', 'homicide', 'jacobs', 'murder', 'j...","[0.4028, 0.3986, 0.3328, 0.3306, 0.3267]",[[-2.9223869e-05 5.0942481e-02 -1.0382515e-02...
2740,abcnews--2019-12-02--Mother charged with murde...,2019-12-02,abcnews,Mother charged with murder in deaths of 2 youn...,the mother of two young children found hanging...,NaN,https://abcnews.go.com/US/wireStory/mother-cha...,"Mon, 02 Dec 2019 11:30:59 -0500",1575304259,1575308811,"crime, law and justice",crime,en,"['autopsy', 'arrest', 'forensic', 'prosecutors...","[0.4117, 0.3691, 0.3408, 0.288, 0.285]",[[ 0.07943991 0.09704047 0.00038995 ... 0.0...
7038,ageofautism--2019-04-12--Physician Father and ...,2019-04-12,ageofautism,"Physician, Father and Caretaker of 29 Year Old...",one family member said derek can be violent a...,Age of Autism,http://feedproxy.google.com/~r/ageofautism/~3/...,2019-04-12 09:00:00+00:00,1555074000,1567543083,"crime, law and justice",crime,en,"['derek', 'rex', 'autism', 'anesthesiologist',...","[0.3437, 0.3433, 0.3345, 0.3082, 0.2677]",[[-0.00611528 0.05226014 -0.01292173 ... 0.0...


In [38]:
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from math import sqrt
import prince
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import umap.umap_ as umap
import numpy as np

In [ ]:
#Split data into Training and Testing
from sklearn.model_selection import train_test_split
train,test = train_test_split(keys_df,test_size=0.20)
X_train=train.loc[:,['Keywords','Scores','Embeddings','key_umap']]
X_test=test.loc[:,['Keywords','Scores','Embeddings','key_umap']]

In [ ]:
#Intialize clustering and partitioning
kmeans = KMeans(n_clusters=7,init='k-means++', max_iter=300, n_init=10) 
cv = KFold(n_splits=5, random_state=42, shuffle=True)

counter=0
overall_CSAI_all_samples=[]

# Now, let's keep only X_train for further partitioning
for train_index, _ in cv.split(X_train):
    X_train_prt = X_train.iloc[train_index]
    #print(X_train_partitioned)
 
    # Convert strings representing lists to actual lists of floats
    #X_train["key_umap"] = X_train["key_umap"].apply(lambda x: [float(val) for val in x.strip('][').split(', ')])
    #X_test["key_umap"] = X_test["key_umap"].apply(lambda x: [float(val) for val in x.strip('][').split(', ')])

    #print(X_train.shape)
    # Take only the scores/features for both testing and training
    X_train_prt=X_train_prt[['Keywords','Scores','Embeddings','key_umap']]
    X_test=X_test[['Keywords','Scores','Embeddings','key_umap']]
    
    # Convert to NumPy array
    embedding_values_array = np.array(X_train_prt["key_umap"].tolist())
    
    # Reduce dimensionality using UMAP
    umap_embedding = umap.UMAP().fit_transform(embedding_values_array)
    
    # Apply KMeans clustering with UMAP
    #kmeans = KMeans(n_clusters=5,init='k-means++', max_iter=300, n_init=10)  # You may adjust the number of clusters as needed
    labels = kmeans.fit_predict(umap_embedding)
    X_train_prt["Cluster"] = [str(label) for label in labels]
    #print(f"Num of clusters: {len(np.unique(labels))}")
    
    # Model fit with Testing data
    embedding_values_array_test = np.array(X_test["key_umap"].tolist())
    # Reduce dimensionality using UMAP
    umap_embedding_test = umap.UMAP().fit_transform(embedding_values_array_test)
    y_pred_test=kmeans.predict(umap_embedding_test)
    X_test["Cluster"] = [str(label) for label in y_pred_test]
    #print(f"Num of clusters: {len(np.unique(labels))}")  

    # Calculate the sum of values in "key_umap" for each row
    X_train_prt['sum_umap_train'] = X_train_prt['key_umap'].apply(np.sum)
    X_test['sum_umap_test'] = X_test['key_umap'].apply(np.sum)

    # Calculate the sum of values in "Scores" for each row
    #X_train['sum_umap_train'] = X_train['Scores'].apply(np.sum)
    #X_test['sum_umap_train'] = X_test['Scores'].apply(np.sum)
    
    # incude the sum of key_umap as new column
    Xtrain=X_train_prt[['Scores','Cluster', 'sum_umap_train']]
    Xtest=X_test[['Scores','Cluster', 'sum_umap_test']]

    #Train :  group and sum values of each cluster on train
    Xtrain=Xtrain.groupby(['Cluster']).sum()
    Xtrain2=Xtrain.T
    Xtrain2.columns=['c0', 'c1', 'c2','c3', 'c4', 'c5', 'c6'];
    Xtrain3=Xtrain2.T
    Xtrain3=Xtrain3["sum_umap_train"];Xtrain3

    # Test: group and sum values of each cluster  on test
    Xtest=Xtest.groupby(['Cluster']).sum()
    Xtest2=Xtest.T
    #Xtest2.columns=['c0', 'c1', 'c2','c3', 'c4'];
    Xtest3=Xtest2.T
    Xtest3=Xtest3["sum_umap_test"];Xtest3
    
    # Train:Total sum of values in all clusters on train
    Xtrain3_sums = Xtrain3.sum(axis=0)
    # Train: Normalize values by dividing the values of each cluster by the total number 
    ct0=pd.Series(Xtrain3.iloc[0]/Xtrain3_sums)
    ct1=pd.Series(Xtrain3.iloc[1]/Xtrain3_sums)
    ct2=pd.Series(Xtrain3.iloc[2]/Xtrain3_sums)
    ct3=pd.Series(Xtrain3.iloc[3]/Xtrain3_sums)
    ct4=pd.Series(Xtrain3.iloc[4]/Xtrain3_sums)
    ct5=pd.Series(Xtrain3.iloc[5]/Xtrain3_sums)
    ct6=pd.Series(Xtrain3.iloc[6]/Xtrain3_sums)
    
    #Join clusters as columns and rename
    df_per_train=pd.concat([ct0,ct1,ct2,ct3,ct4,ct5,ct6],axis=1)
    df_per_train.columns = ['c0', 'c1', 'c2','c3', 'c4', 'c5', 'c6']
    #df_per_test=df_per_train.T
    df_per_train

   
    # Test: Total sum of values in all clusters on Test
    Xtest3_sums = Xtest3.sum(axis=0)
    # Normalize values by dividing the values of each cluster by the total number 
    c00=pd.Series(Xtest3.iloc[0]/Xtest3_sums)
    c10=pd.Series(Xtest3.iloc[1]/Xtest3_sums)
    c20=pd.Series(Xtest3.iloc[2]/Xtest3_sums)
    c30=pd.Series(Xtest3.iloc[3]/Xtest3_sums)
    c40=pd.Series(Xtest3.iloc[4]/Xtest3_sums)
    c50=pd.Series(Xtest3.iloc[5]/Xtest3_sums)
    c60=pd.Series(Xtest3.iloc[6]/Xtest3_sums)

    #Join clusters as columns and rename
    df_per_test=pd.concat([c00,c10,c20,c30,c40,c50,c60],axis=1)
    df_per_test.columns = ['c0', 'c1', 'c2','c3', 'c4', 'c5', 'c6']
    #df_per_test=df_per_train.T
    df_per_test

    # Define overall RMSE variable before the loop
   
  
   # Calculate the range of the values  for train
    range_y = np.max(df_per_train) - np.min(df_per_train)
    overall_rmse_per_cluster = []
    
    # Calculate RMSE for each cluster 
    for i in range(3):
        #train_df = train_dfs[i]
        #test_df = test_dfs[i]
        
        # Calculate the squared errors for each cluster in each partition
        #squared_errors = (sums_df1_cluster.values - sums_df2_cluster.values) ** 2
        squared_errors = (df_per_train.iloc[:, i] - df_per_test.iloc[:, i]) ** 2
        # Calculate the mean squared error for each cluster in each partition
        mean_squared_error = np.mean(squared_errors)
        # Calculate the root mean squared error (RMSE) for each cluster in each partition
        rmse_per_cluster = np.sqrt(mean_squared_error)/range_y
        #print(f"RMSE for partition {i+1} for each cluster:", rmse_per_cluster)
        # Append the RMSE for this partition to the overall RMSE list
        overall_rmse_per_cluster.append(rmse_per_cluster)
        
        
    print("Sample:",counter)
    print("Training set:",X_train.shape)
    print(f"Num of clusters: {len(np.unique(labels))}") 
    counter += 1
        
    # Convert the overall RMSE list to a numpy array
    overall_rmse_per_cluster = np.array(overall_rmse_per_cluster)
    
    # Calculate the overall RMSE by averaging the RMSEs of all clusters across all partitions
    overall_rmse = np.mean(overall_rmse_per_cluster)
    overall_CSAI_all_samples.append(overall_rmse)
    
    #print("\nOverall RMSE for each cluster:", overall_rmse_per_cluster)
    #print("\nOverall RMSE across all clusters and partitions:", overall_rmse)
    print("\n CSAI per cluster in this partition:", [round(val, 4) for val in overall_rmse_per_cluster])
    print("\n  CSAI  in this  partition :", round(overall_rmse, 4))
    print("\n")

overall_CSAI = np.mean(overall_CSAI_all_samples)
print("\nOverall CSAI across all samples :", round(overall_CSAI, 4))